In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "2g").\
        getOrCreate()

22/02/17 16:57:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
df = spark.read.option("header", "true").csv("data/final_dataset.csv")
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: string (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: string (nullable = true)
 |-- Protocol: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: string (nullable = true)
 |-- Tot Fwd Pkts: string (nullable = true)
 |-- Tot Bwd Pkts: string (nullable = true)
 |-- TotLen Fwd Pkts: string (nullable = true)
 |-- TotLen Bwd Pkts: string (nullable = true)
 |-- Fwd Pkt Len Max: string (nullable = true)
 |-- Fwd Pkt Len Min: string (nullable = true)
 |-- Fwd Pkt Len Mean: string (nullable = true)
 |-- Fwd Pkt Len Std: string (nullable = true)
 |-- Bwd Pkt Len Max: string (nullable = true)
 |-- Bwd Pkt Len Min: string (nullable = true)
 |-- Bwd Pkt Len Mean: string (nullable = true)
 |-- Bwd Pkt Len Std: string (nullable = true)
 |-- Flow Byts/s: string (nullable = true)
 |-- Flow Pkts/s: string (nullable =

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="hour", outputCol="result")

result = discretizer.fit(df).transform(df)
result.show()

In [3]:
from pyspark.sql.functions import split, explode, col, desc
from pyspark.ml import Pipeline 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml.feature import StringIndexer 
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexers = [
StringIndexer(inputCol="Dst IP", outputCol = "dstip_index"),
StringIndexer(inputCol="FIN Flag Cnt", outputCol = "fin_index"),
StringIndexer(inputCol="SYN Flag Cnt", outputCol = "syn_index"),
StringIndexer(inputCol="URG Flag Cnt", outputCol = "urg_index"),
StringIndexer(inputCol="PSH Flag Cnt", outputCol = "cwe_index"),
StringIndexer(inputCol="ECE Flag Cnt", outputCol = "ece_index"),
StringIndexer(inputCol="Label", outputCol = "label")]

In [4]:
pipeline = Pipeline(stages=indexers)
df = df.select("Dst IP", 
               "FIN Flag Cnt", 
               "SYN Flag Cnt", 
               "URG Flag Cnt",
               "PSH Flag Cnt",
               "ECE Flag Cnt",
               "Label"
              )
#df = df.sample(fraction=0.3)
indexed_rows_df = pipeline.fit(df).transform(df)
indexed_rows_df.show(5, False)

22/02/17 16:57:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/02/17 17:04:14 WARN DAGScheduler: Broadcasting large task binary with size 1284.0 KiB


+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+
|Dst IP      |FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index|
+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+
|203.73.24.75|0           |1           |0           |0           |0           |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |
|203.73.24.75|0           |0           |0           |0           |0           |0.0  |25.0       |0.0      |0.0      |0.0      |0.0      |0.0      |
|203.73.24.75|0           |1           |0           |0           |0           |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |
|203.73.24.75|0           |0           |0           |0           |0           |0.0  |25.0       |0.0      |0.0  

In [5]:
vectorAssembler = VectorAssembler(inputCols = ["dstip_index","fin_index", "syn_index", "urg_index", "cwe_index","ece_index"],outputCol = "features")
vindexed_rows_df = vectorAssembler.transform(indexed_rows_df)
vindexed_rows_df.show(5, False)

22/02/17 17:04:40 WARN DAGScheduler: Broadcasting large task binary with size 1290.9 KiB


+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+--------------------+
|Dst IP      |FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index|features            |
+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+--------------------+
|203.73.24.75|0           |1           |0           |0           |0           |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |(6,[0,2],[25.0,1.0])|
|203.73.24.75|0           |0           |0           |0           |0           |0.0  |25.0       |0.0      |0.0      |0.0      |0.0      |0.0      |(6,[0],[25.0])      |
|203.73.24.75|0           |1           |0           |0           |0           |0.0  |25.0       |0.0      |1.0      |0.0      |0.0      |0.0      |(6,[0,2]

In [6]:
splits = vindexed_rows_df.randomSplit([0.6,0.4], 42) 
# optional value 42 is seed for sampling 
train_df = splits[0] 
test_df = splits[1]

In [7]:
nb = NaiveBayes(modelType="multinomial")
nbmodel = nb.fit(train_df)

22/02/17 17:04:46 WARN DAGScheduler: Broadcasting large task binary with size 1323.7 KiB
22/02/17 17:06:07 WARN DAGScheduler: Broadcasting large task binary with size 1302.5 KiB


In [8]:
predictions_df = nbmodel.transform(test_df)
predictions_df.show(5, True)

22/02/17 17:08:40 WARN DAGScheduler: Broadcasting large task binary with size 1327.4 KiB


+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+---------+--------------------+--------------------+----------+
|      Dst IP|FIN Flag Cnt|SYN Flag Cnt|URG Flag Cnt|PSH Flag Cnt|ECE Flag Cnt|label|dstip_index|fin_index|syn_index|urg_index|cwe_index|ece_index| features|       rawPrediction|         probability|prediction|
+------------+------------+------------+------------+------------+------------+-----+-----------+---------+---------+---------+---------+---------+---------+--------------------+--------------------+----------+
|172.31.69.25|           0|           0|           0|           0|           0|  0.0|        0.0|      0.0|      0.0|      0.0|      0.0|      0.0|(6,[],[])|[-0.6812565473673...|[0.50598080390174...|       0.0|
|172.31.69.25|           0|           0|           0|           0|           0|  0.0|        0.0|      0.0|      0.0|      0.0|      0.0|      0.0|(6,[],[])

In [9]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy") 
nbaccuracy = evaluator.evaluate(predictions_df) 
print("Test accuracy = " + str(nbaccuracy))

22/02/17 17:08:47 WARN DAGScheduler: Broadcasting large task binary with size 1324.5 KiB


Test accuracy = 0.7389105148836407
